Cell 1 – Imports and basic config

In [22]:
import os
import random
import json
from collections import Counter

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_recall_fscore_support,
    confusion_matrix,
    classification_report
)

import matplotlib.pyplot as plt

# Reproducibility helper
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


Cell 2 – Paths & config

In [23]:
# Adjust these paths if needed
PROFILE_FEATURES_CSV = "./twibot_user_features_clean.csv"   # contains profile + label + split
GRAPH_FEATURES_CSV   = "./graph_features.csv"               # contains graph features per user id

# Output artifact paths
SCALER_PATH          = "./twibot_scaler.pkl"
ENSEMBLE_CKPT_PATH   = "./twibot_mlp_ensemble.pt"
RESULTS_JSON_PATH    = "./twibot_mlp_results.json"

# Best architecture we chose from previous experiments
ARCHITECTURE = "mlp_256_128_64"
ENSEMBLE_SEEDS = [42, 7, 2024]

print("Architecture:", ARCHITECTURE)
print("Ensemble seeds:", ENSEMBLE_SEEDS)

Architecture: mlp_256_128_64
Ensemble seeds: [42, 7, 2024]


Cell 3 – Load data & basic EDA

In [24]:
# Load profile features (already has label + split columns)
profile_df = pd.read_csv(PROFILE_FEATURES_CSV)
print("[*] Profile features shape:", profile_df.shape)
print(profile_df.head())

# Optional: quick label + split distribution from this single merged CSV
print("\nLabel distribution (from profile_df):")
print(profile_df["label"].value_counts())

print("\nSplit distribution (from profile_df):")
print(profile_df["split"].value_counts())

# Load graph features (must share 'id' column with profile_df)
graph_df = pd.read_csv(GRAPH_FEATURES_CSV)
print("\n[*] Graph features shape:", graph_df.shape)
print(graph_df.head())

# Merge on 'id' to build final feature dataframe
feat_df = profile_df.merge(graph_df, on="id", how="inner")
print("\n[*] After merging profile + graph, shape:", feat_df.shape)

# Keep only rows with split in {train, val, test} for safety
feat_df = feat_df[feat_df["split"].isin(["train", "val", "test"])].copy()

print("\n[*] Final available splits after merge:")
print(feat_df["split"].value_counts())

# Show basic stats for first few numeric columns
print("\n[*] Basic stats preview (first N numeric columns):")
print(feat_df.describe().iloc[:, :10])


[*] Profile features shape: (990913, 18)
   followers_count  following_count  tweet_count   ff_ratio  \
0             7316              215         3098  33.870370   
1              123             1090         1823   0.112741   
2                3               62           66   0.047619   
3              350              577          237   0.605536   
4              240              297         3713   0.805369   

   tweets_per_following  tweets_per_follower  desc_length  has_url  verified  \
0             14.342593             0.423398           92        1         0   
1              1.670944            14.701613           10        0         0   
2              1.047619            16.500000            1        0         0   
3              0.410035             0.675214          127        0         0   
4             12.459732            15.406639           35        0         0   

   account_age_days  log_followers_count  log_following_count  \
0       2132.706977             8.

Cell 4 – Prepare features, labels, and split

In [25]:
# Columns that are NOT features
NON_FEATURE_COLS = ["id", "label", "split"]

# Automatically infer feature columns
feature_cols = [c for c in feat_df.columns if c not in NON_FEATURE_COLS]
print("\n[*] Final feature columns ({}):".format(len(feature_cols)))
print(feature_cols)

# Encode labels: assume 'bot' vs 'human'
label_map = {"human": 0, "bot": 1}
feat_df["y"] = feat_df["label"].map(label_map)

# Sanity check
print("\n[*] Label mapping check:")
print(feat_df[["label", "y"]].value_counts().head())

# Split by 'split' column
train_df = feat_df[feat_df["split"] == "train"].copy()
val_df   = feat_df[feat_df["split"] == "val"].copy()
test_df  = feat_df[feat_df["split"] == "test"].copy()

print("\n[*] Split sizes:")
print("  train:", train_df.shape)
print("  val  :", val_df.shape)
print("  test :", test_df.shape)



[*] Final feature columns (26):
['followers_count', 'following_count', 'tweet_count', 'ff_ratio', 'tweets_per_following', 'tweets_per_follower', 'desc_length', 'has_url', 'verified', 'account_age_days', 'log_followers_count', 'log_following_count', 'log_tweet_count', 'log_listed_count', 'account_age_years', 'out_following', 'in_following', 'out_followers', 'in_followers', 'out_mentioned', 'in_mentioned', 'out_retweeted', 'in_retweeted', 'deg_follow', 'deg_mention', 'deg_retweet']

[*] Label mapping check:
label  y
human  0    854940
bot    1    135973
Name: count, dtype: int64

[*] Split sizes:
  train: (692925, 30)
  val  : (199108, 30)
  test : (98880, 30)


Cell 5 – Handle class imbalance (undersampling + pos_weight)

In [26]:
# class imbalance handling (undersample majority class + pos_weight)

# Count class distribution in raw train split
train_counts = train_df["y"].value_counts()
num_humans = int(train_counts.get(0, 0))
num_bots   = int(train_counts.get(1, 0))

print("\n[*] Raw TRAIN label distribution (before rebalancing):")
print(train_counts)

# Simple strategy: undersample humans to ~2x bots (same as earlier code)
# If bots=~51k, humans_sampled=~103k
target_humans = min(num_humans, 2 * num_bots)

humans_df = train_df[train_df["y"] == 0]
bots_df   = train_df[train_df["y"] == 1]

humans_sampled = humans_df.sample(n=target_humans, random_state=42)
train_rebalanced_df = pd.concat([humans_sampled, bots_df], axis=0).sample(frac=1.0, random_state=42)

print("\n[*] TRAIN label distribution AFTER rebalancing:")
print(train_rebalanced_df["y"].value_counts())

# Build X, y for train/val/test
X_train = train_rebalanced_df[feature_cols].values
y_train = train_rebalanced_df["y"].values

X_val   = val_df[feature_cols].values
y_val   = val_df["y"].values

X_test  = test_df[feature_cols].values
y_test  = test_df["y"].values

print("\n[*] Shapes before scaling:")
print("  X_train:", X_train.shape, "y_train:", y_train.shape)
print("  X_val  :", X_val.shape,   "y_val  :", y_val.shape)
print("  X_test :", X_test.shape,  "y_test :", y_test.shape)

# pos_weight = (#neg / #pos) = humans / bots (on rebalanced train)
num_humans_reb = (y_train == 0).sum()
num_bots_reb   = (y_train == 1).sum()

pos_weight_value = num_humans_reb / num_bots_reb
print("\n[*] pos_weight (for bot class) in BCEWithLogitsLoss:", round(pos_weight_value, 4))



[*] Raw TRAIN label distribution (before rebalancing):
y
0    641040
1     51885
Name: count, dtype: int64

[*] TRAIN label distribution AFTER rebalancing:
y
0    103770
1     51885
Name: count, dtype: int64

[*] Shapes before scaling:
  X_train: (155655, 26) y_train: (155655,)
  X_val  : (199108, 26) y_val  : (199108,)
  X_test : (98880, 26) y_test : (98880,)

[*] pos_weight (for bot class) in BCEWithLogitsLoss: 2.0


Cell 6 – Standardize features

In [ ]:
# Cell 6: standard scaling (fit on TRAIN only)

import joblib 

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

print("\n[*] Scaling done (StandardScaler fitted on TRAIN).")

# Save scaler for later use (deployment / demo)
joblib.dump(scaler, SCALER_PATH)
print("[*] Scaler saved to:", SCALER_PATH)



[*] Scaling done (StandardScaler fitted on TRAIN).
[*] Scaler saved to: ./twibot_scaler.pkl


Cell 7 – MLP model, train loop, eval helpers

In [16]:
# Cell 7: Define MLP model, training loop, and evaluation utilities

class MLP(nn.Module):
    """Flexible MLP with given hidden_dims list."""
    def __init__(self, input_dim, hidden_dims, dropout=0.2):
        super().__init__()
        layers = []
        last_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(last_dim, h))
            layers.append(nn.BatchNorm1d(h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            last_dim = h
        layers.append(nn.Linear(last_dim, 1))  # binary logit output
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze(-1)


def train_one_model(hidden_dims,
                    seed=42,
                    lr=1e-3,
                    max_epochs=60,
                    patience=10,
                    batch_size=4096):
    """Train one MLP with a given hidden_dims and random seed."""
    set_seed(seed)

    train_loader, val_loader, test_loader = make_dataloaders(batch_size=batch_size)
    input_dim = X_train_scaled.shape[1]

    model = MLP(input_dim=input_dim, hidden_dims=hidden_dims).to(DEVICE)

    # pos_weight for bots (label 1) to handle imbalance
    num_pos = (y_train == 1).sum()
    num_neg = (y_train == 0).sum()
    pos_weight = torch.tensor(num_neg / num_pos, dtype=torch.float32, device=DEVICE)

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val_f1 = -1.0
    best_state = None
    epochs_no_improve = 0

    for epoch in range(1, max_epochs + 1):
        # ----- Train -----
        model.train()
        train_losses = []
        for xb, yb in train_loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE).float()  # BCEWithLogitsLoss expects float for targets

            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())

        avg_train_loss = float(np.mean(train_losses))

        # ----- Validate -----
        model.eval()
        all_val_logits = []
        all_val_labels = []
        with torch.no_grad():
            for xb, yb in val_loader:
                xb = xb.to(DEVICE)
                logits = model(xb)
                all_val_logits.append(logits.cpu().numpy())
                all_val_labels.append(yb.numpy())

        val_logits = np.concatenate(all_val_logits)
        val_labels = np.concatenate(all_val_labels)

        # Default threshold 0.5 for early stopping metric
        val_probs = 1.0 / (1.0 + np.exp(-val_logits))
        val_preds = (val_probs >= 0.5).astype(int)

        val_acc = accuracy_score(val_labels, val_preds)
        val_macro_f1 = f1_score(val_labels, val_preds, average="macro")

        print(f"Epoch {epoch:02d} | TrainLoss={avg_train_loss:.4f} | "
              f"ValAcc={val_acc:.4f} | ValMacroF1={val_macro_f1:.4f}")

        # Track best model by VAL macro F1
        if val_macro_f1 > best_val_f1 + 1e-4:
            best_val_f1 = val_macro_f1
            best_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            print(f"[*] Early stopping triggered at epoch {epoch}.")
            break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)

    return model, best_val_f1


Cell 8 – Threshold tuning + final evaluation helpers

In [17]:
# Cell 8: Threshold search on VAL and final TEST evaluation

def find_best_threshold(model, val_loader, thresholds=None):
    """Search threshold on validation to maximize macro F1."""
    if thresholds is None:
        thresholds = np.linspace(0.5, 0.9, 41)  # 0.50, 0.51, ..., 0.90

    model.eval()
    all_logits = []
    all_labels = []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            all_logits.append(logits.cpu().numpy())
            all_labels.append(yb.numpy())

    logits = np.concatenate(all_logits)
    labels = np.concatenate(all_labels)
    probs = 1.0 / (1.0 + np.exp(-logits))

    best_thr = 0.5
    best_f1 = -1.0

    for thr in thresholds:
        preds = (probs >= thr).astype(int)
        f1 = f1_score(labels, preds, average="macro")
        if f1 > best_f1:
            best_f1 = f1
            best_thr = thr

    return best_thr, best_f1


def evaluate_on_loader(model, loader, threshold):
    """Evaluate model on a DataLoader using given threshold."""
    model.eval()
    all_logits = []
    all_labels = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            logits = model(xb)
            all_logits.append(logits.cpu().numpy())
            all_labels.append(yb.numpy())

    logits = np.concatenate(all_logits)
    labels = np.concatenate(all_labels)
    probs = 1.0 / (1.0 + np.exp(-logits))
    preds = (probs >= threshold).astype(int)

    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average="macro")
    report = classification_report(labels, preds, target_names=["Human", "Bot"])

    return acc, macro_f1, report


Cell 9 – Train with best architecture (mlp_256_128_64) using 3 seeds and simple ensemble

In [18]:
# Cell 9: Train best architecture (mlp_256_128_64) with multiple seeds and ensemble

BEST_ARCH_NAME = "mlp_256_128_64"
BEST_HIDDEN_DIMS = [256, 128, 64]
SEEDS = [42, 7, 2024]

print(f"===========================================")
print(f" Training best architecture: {BEST_ARCH_NAME}")
print(f"===========================================")

models = []
val_f1_per_seed = {}

for seed in SEEDS:
    print(f"\n============================")
    print(f" Training model with seed={seed}")
    print(f"============================")
    model, best_val_f1 = train_one_model(
        hidden_dims=BEST_HIDDEN_DIMS,
        seed=seed,
        lr=1e-3,
        max_epochs=60,
        patience=10,
        batch_size=4096,
    )
    models.append(model)
    val_f1_per_seed[seed] = best_val_f1
    print(f"[*] Best VAL macro F1 for seed {seed}: {best_val_f1:.4f}")

print("\n[*] VAL macro F1 per seed:", val_f1_per_seed)

# Build a simple ensemble that averages logits from all 3 models
class EnsembleWrapper(nn.Module):
    def __init__(self, models):
        super().__init__()
        self.models = nn.ModuleList(models)

    def forward(self, x):
        logits = []
        for m in self.models:
            logits.append(m(x))
        stacked = torch.stack(logits, dim=0)  # [n_models, batch_size]
        return stacked.mean(dim=0)  # average logits


# Freeze ensemble models (no further training)
for m in models:
    m.eval()
ensemble_model = EnsembleWrapper(models).to(DEVICE)

# Get loaders
_, val_loader, test_loader = make_dataloaders(batch_size=4096)

# Threshold search on VAL for ensemble
best_thr, best_val_macro_f1 = find_best_threshold(ensemble_model, val_loader)
print(f"\n[*] Best threshold on VAL (macro F1): {best_thr:.4f}")
print(f"[*] Best VAL macro F1 at that threshold: {best_val_macro_f1:.4f}")

# Final evaluation on TEST
test_acc, test_macro_f1, test_report = evaluate_on_loader(
    ensemble_model, test_loader, threshold=best_thr
)

print(f"\nTEST Accuracy: {test_acc:.4f} (threshold={best_thr:.4f})")
print(f"TEST Macro F1: {test_macro_f1:.4f}")
print("\nClassification report (TEST):")
print(test_report)

# Optional: Save summary JSON for your paper/report
results_summary = {
    "architecture": BEST_ARCH_NAME,
    "hidden_dims": BEST_HIDDEN_DIMS,
    "seeds": SEEDS,
    "val_macro_f1_per_seed": val_f1_per_seed,
    "best_val_macro_f1_ensemble": float(best_val_macro_f1),
    "best_threshold": float(best_thr),
    "test_accuracy": float(test_acc),
    "test_macro_f1": float(test_macro_f1),
}

out_path = os.path.join(DATA_DIR, "twibot_best_mlp_results.json")
with open(out_path, "w") as f:
    json.dump(results_summary, f, indent=2)

print(f"\n[*] Saved summary to: {out_path}")


 Training best architecture: mlp_256_128_64

 Training model with seed=42
Epoch 01 | TrainLoss=0.6768 | ValAcc=0.5596 | ValMacroF1=0.5531
Epoch 02 | TrainLoss=0.6669 | ValAcc=0.5614 | ValMacroF1=0.5549
Epoch 03 | TrainLoss=0.6161 | ValAcc=0.5445 | ValMacroF1=0.5405
Epoch 04 | TrainLoss=0.6244 | ValAcc=0.5612 | ValMacroF1=0.5544
Epoch 05 | TrainLoss=0.6113 | ValAcc=0.5577 | ValMacroF1=0.5514
Epoch 06 | TrainLoss=0.6140 | ValAcc=0.5542 | ValMacroF1=0.5487
Epoch 07 | TrainLoss=0.6216 | ValAcc=0.5628 | ValMacroF1=0.5557
Epoch 08 | TrainLoss=0.6095 | ValAcc=0.5675 | ValMacroF1=0.5597
Epoch 09 | TrainLoss=0.6080 | ValAcc=0.5500 | ValMacroF1=0.5452
Epoch 10 | TrainLoss=0.6061 | ValAcc=0.5712 | ValMacroF1=0.5625
Epoch 11 | TrainLoss=0.6090 | ValAcc=0.5575 | ValMacroF1=0.5516
Epoch 12 | TrainLoss=0.6079 | ValAcc=0.5944 | ValMacroF1=0.5803
Epoch 13 | TrainLoss=0.6102 | ValAcc=0.5621 | ValMacroF1=0.5550
Epoch 14 | TrainLoss=0.6141 | ValAcc=0.5502 | ValMacroF1=0.5454
Epoch 15 | TrainLoss=0.6093 | 

In [19]:
# Cell: helper to get probabilities and predictions from the ensemble

def ensemble_predict_proba(models_dict, X_np, batch_size=4096):
    """Return averaged bot probability from ensemble for given features."""
    models = list(models_dict.values())
    for m in models:
        m.eval()

    X_tensor = torch.from_numpy(X_np).float().to(device)
    probs_list = []

    with torch.no_grad():
        for m in models:
            logits = []
            for i in range(0, X_tensor.size(0), batch_size):
                batch = X_tensor[i:i+batch_size]
                batch_logits = m(batch).squeeze(1)
                logits.append(batch_logits.cpu().numpy())
            logits = np.concatenate(logits)
            probs = 1 / (1 + np.exp(-logits))  # sigmoid
            probs_list.append(probs)

    avg_probs = np.mean(np.stack(probs_list, axis=0), axis=0)
    return avg_probs  # probability of class "bot"


In [20]:
# Cell: confusion matrices and classification reports
from sklearn.metrics import confusion_matrix, classification_report

# VAL
val_bot_probs = ensemble_predict_proba(best_models, X_val)
val_preds = (val_bot_probs >= best_threshold).astype(int)  # 1=bot, 0=human

print("=== VALIDATION SET ===")
print(classification_report(y_val, val_preds, target_names=["Human", "Bot"]))

cm_val = confusion_matrix(y_val, val_preds)

# TEST
test_bot_probs = ensemble_predict_proba(best_models, X_test)
test_preds = (test_bot_probs >= best_threshold).astype(int)

print("\n=== TEST SET ===")
print(classification_report(y_test, test_preds, target_names=["Human", "Bot"]))

cm_test = confusion_matrix(y_test, test_preds)

print("\nVAL Confusion Matrix:\n", cm_val)
print("\nTEST Confusion Matrix:\n", cm_test)


NameError: name 'best_models' is not defined